# This is Henry's fonts as data homework for Material of Language class

I was not very creative with homework 4, I just play with the example code, and made sketches around it ... 
 

In [1]:
import json

In [2]:
all_data = json.load(open("./hersheytext.json"))


In [3]:
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy

In [4]:
from numpy.random import uniform, normal, choice

In [5]:
all_data.keys()

dict_keys(['astrology', 'cursive', 'cyrillic', 'futural', 'futuram', 'gothiceng', 'gothicger', 'gothicita', 'greek', 'japanese', 'markers', 'mathlow', 'mathupp', 'meteorology', 'music', 'scriptc', 'scripts', 'symbolic', 'timesg', 'timesi', 'timesib', 'timesr', 'timesrb'])

In [6]:
futural = all_data['futural']['chars']

In [7]:
def idx(ch):
    return ord(ch) - 33

In [8]:
futural[idx('g')]

{'d': 'M15,8 L15,24 14,27 13,28 11,29 8,29 6,28 M15,11 L13,9 11,8 8,8 6,9 4,11 3,14 3,16 4,19 6,21 8,22 11,22 13,21 15,19',
 'o': 10}

In [9]:
def dch(font, ch):
    if ch == ' ':
        return 'M8,0'
    else:
        return font[idx(ch)]['d']
print(dch(futural, 'a'))
print(dch(futural, ' '))

M15,8 L15,22 M15,11 L13,9 11,8 8,8 6,9 4,11 3,14 3,16 4,19 6,21 8,22 11,22 13,21 15,19
M8,0


In [10]:
from flat import command
from bezmerizing import Path

In [11]:
import numpy as np
def topath(font, ch):
    cmds = []
    mode = ''
    for t in dch(font, ch).split():
        if t[0] in ('M', 'L'):
            mode = t[0]
            t = t[1:]
        coords = [int(x) for x in t.split(',')]
        if mode == 'M':
            cmds.append(command.moveto(*coords))
        else:
            cmds.append(command.lineto(*coords))
    return Path(cmds)

In [12]:
from flat import document, shape, rgb
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

In [13]:
amp_path = topath(futural, '&')
page = document(30, 20, 'mm').addpage()
pen = shape().stroke(rgb(0, 0, 0)).width(2)
page.place(pen.path(amp_path.scalexy(0.5, 0.75)))
show(page)

In [14]:

from itertools import chain
def strtopath(font, s, kern=1):
    current_x = 0
    all_paths = []
    for ch in s:
        path = topath(font, ch).translate(current_x, 0)
        all_paths.append(path)
        x_extent = max([c.x for c in path.commands])
        current_x = x_extent + kern
    return Path(list(chain(*all_paths)))

In [15]:
str_path = strtopath(futural, "Quartzy foxes").scale(0.5)
page = document(100, 16, 'mm').addpage()
pen = shape().stroke(rgb(0, 0, 0)).width(2)
page.place(pen.path(str_path))
show(page)

In [16]:
def strtopath_wrap(font, s, kern=1, lineheight=32, width=360, height=360):
    current = [0, 0]
    all_paths = []
    for word in s.split():
        wpath = strtopath(font, word, kern)
        wpath_width = max([c.x for c in wpath.commands])
        if current[0] + wpath_width > width:
            current[1] += lineheight
            current[0] = 0
        if current[1] > height - lineheight:
            break
        all_paths.append(wpath.translate(current[0], current[1]))
        current[0] += wpath_width + 8
    return Path(list(chain(*all_paths)))

In [17]:
src = '''\
The threat to linguistic authority made by the manipulation
of the words on the page was that it returned the written
language to the specific place, instance, conditions of
production: it became a highly marked text. The unmarked
text... appeared, as it were, to "speak itself."'''

In [18]:
doc_height = 360 # points
doc_width = 360 # points
d = document(doc_width, doc_height, 'pt')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(2)
text_path = strtopath_wrap(futural, src).translate(24, 24)
page.place(line_fig.path(text_path))
show(page)

In [19]:
import sys
!{sys.executable} -m pip install opensimplex

In [20]:
import opensimplex

In [21]:
noisy = opensimplex.OpenSimplex()

In [22]:
text_path = strtopath_wrap(all_data['scriptc']['chars'], src, kern=0)
doc_height = 240 # points
doc_width = 240 # points
d = document(doc_width, doc_height, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(3).join('round').cap('round')
for i in range(1):
    manip = []
    for cmd in text_path:
        nx = noisy.noise3d(cmd.x*0.02, cmd.y*0.02, 20-(100*i*5.02)) * 6
        ny = noisy.noise3d(cmd.x*0.52, cmd.y*0.02, 100+(100*i*10.02)) * 3
        cmd.x += nx
        cmd.y += ny
    manip_tr = text_path.scale(0.75).translate(24, 24)
    this_text = line_fig.path(manip_tr)
    page.place(this_text)
show(page)

In [23]:
text_path = strtopath_wrap(all_data['scriptc']['chars'], src, kern=1)
doc_height = 240 # points
doc_width = 240 # points
d = document(doc_width, doc_height, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(0.9).join('miter').cap('square')
for i in range(1):
    manip = []
    for cmd in text_path:
        nx = noisy.noise3d(cmd.x*0.02, cmd.y*0.02, 100*i*0.02) * 10.2
        ny = noisy.noise3d(cmd.x*10.2, cmd.y*0.02, 11+(100*i*0.02)) * 11.2
        nz = noisy.noise2d(cmd.x*0.02,100*i*2.5) * 0.2
        cmd.x += nx + nz
        cmd.y += ny + nz
    manip_tr = text_path.scale(0.77).translate(24, 24)
    this_text = line_fig.path(manip_tr)
    page.place(this_text)
show(page)

In [24]:
text_path = strtopath_wrap(all_data['scriptc']['chars'], src, kern=1)
doc_height = 240 # points
doc_width = 240 # points
d = document(doc_width, doc_height, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(2).join('miter').cap('square')
for i in range(1):
    manip = []
    for cmd in text_path:
        nx = noisy.noise2d(cmd.x*0.02, cmd.y*0.02,) * 0.2
        ny = noisy.noise2d(cmd.x*0.12, cmd.y*0.32,) * 2.2
        cmd.x += nx
        cmd.y += ny
    manip_tr = text_path.scale(0.77).translate(24, 24)
    this_text = line_fig.path(manip_tr)
    page.place(this_text)
show(page)

In [25]:
text_path = strtopath_wrap(all_data['scriptc']['chars'], src, kern=1)
doc_height = 240 # points
doc_width = 240 # points
d = document(doc_width, doc_height, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgb(0, 0, 0)).width(4).join('miter').cap('square')
for i in range(1):
    manip = []
    for cmd in text_path:
        nx = noisy.noise4d(cmd.x*0.02, cmd.y*0.02, 100*i*0.02, 120*i*0.2) * 20.2
        ny = noisy.noise4d(cmd.x*0.02, cmd.y*0.02, 100+(100*i*0.02),12) * 123.02
        cmd.x += nx
        cmd.y += ny
    manip_tr = text_path.scale(0.55).translate(24, 24)
    this_text = line_fig.path(manip_tr)
    page.place(this_text)
show(page)

In [26]:
f = font.open("./NotoSans-Regular.ttf")

In [27]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

In [28]:
s = "Apple"

In [29]:
page = document(300, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = glyphcommands(f, ch).scale(factor).translate(cx, 60)
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

In [30]:
page = document(300, 300, 'pt').addpage()
pen = shape().stroke(rgba(255, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = (glyphcommands(f, ch)
                  .scale(factor * uniform(0.75, 4.5))
                  .rotate(i*0.1)
                  .translate(factor + uniform(300), factor + uniform(300)))
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

In [32]:
from itertools import chain
def combine_path(f, s):
    text_paths = []
    cx = 0
    for ch in s:
        glyph_path = glyphcommands(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

In [33]:
from math import sqrt
d = document(350, 72, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(0, 0, 0, 64))
factor = 72 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "quartzy foxes").scale(factor).translate(0, 72)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.5
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [34]:
from math import sqrt
d = document(880, 72, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(230, 0, 50, 100))
factor = 72 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "Peace and Love").scale(factor).translate(0, 75)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.7
        page.place(pen.circle((cx + cmd.x) * 4.13 / 2, (cy + cmd.y) / 2, radius*2))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [35]:
from math import sqrt
d = document(880, 72, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(230, 0, 50, 120))
factor = 72 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "Stay Home").scale(factor).translate(0, 75)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.55
        page.place(pen.circle((cx + cmd.x) * 2.3 / 2, (cy + cmd.y) / 2, radius*2))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [36]:
from math import sqrt
d = document(880, 80, 'mm')
page = d.addpage()
pen = shape().nostroke().fill(rgba(255, 0, 50, 120))
factor = 72 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "RED").scale(factor).translate(0, 70)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.55
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius*1.2))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [37]:
width = 660 # 5in
height = 800 # 4in
border = 9 # 1/8in
lineheight = 15 # size of each line
rectsize = 12 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border  
xoff = border
for line in open("letters.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 255 - (path_len * 3)
        advance = advancefor(f, ch) / f.density * lineheight
        brush = shape().nostroke().fill(rgb(val, val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)
    xoff = border
    yoff += lineheight
show(page)

In [38]:
width = 300 # 5in
height = 800 # 4in
border = 2 # 1/8in
lineheight = 15 # size of each line
rectsize = 12 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border  
xoff = border
for line in open("letters.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 280 - (path_len * 2.2)
        advance = advancefor(f, ch) / f.density * lineheight
        brush = shape().nostroke().fill(rgb(val, val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)
    xoff = border
    yoff += lineheight
show(page)